In [1]:
# !pip install sentence_transformers
# !pip install git+https://github.com/huggingface/transformers.git

In [1]:
import os
os.chdir("..")
print(os.listdir())
from utils.utils import notebook_line_magic
notebook_line_magic()

['.DS_Store', 'utils', 'data', 'notebooks']
Line Magic Set


### Data Exploration 
- The nlp dataset has vector of inconsistent lenghts.
- Couple of options (among many):
- Since each integer represents a token. Each vector can be padded with an unused integer to make all vector have a consistent lenght -> then do PCA if needed to reduce dimensions.
- Use a pretrained encoder to generate embeddings.


## data processing
- manually add padding and use sentence transformer to generate embeddings
- use auto tokenize with padding
- manually add padding and then do tf-idf

## modeling ideas
- create a dataloader with a defined bacth size
- use the dataloader to create embeddings and then merge with the UniqueID
- combined embeddings with other features
- build an XGBoost model and a custom torch fully connected network
- you can als get fancy with concatenating the bert model above with a torch classifier 
- knn classifier might also work well with features and embeddings

In [ ]:
# add padding to the tokens
import ast
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from utils.data_utils import get_data_df, get_padded_tokens, generate_embeddings_from_tokens